In [1]:
import numpy as np
import torch
import opt_einsum
import itertools

import process_matrices
import utils

In [2]:
q_s = 1
q_c = 2

In [3]:
# Generalized Amplitude Damping Krauss Operators
N = 0
g = 0.5

K_1 = torch.tensor([[np.sqrt(1 - N), 0],[0, np.sqrt(1 - N) * np.sqrt(1 - g)]])
K_2 = torch.tensor([[0,np.sqrt(g*(1-N))],[0,0]])
K_3 = torch.tensor([[np.sqrt(N)*np.sqrt(1-g), 0],[0,np.sqrt(N)]])
K_4 = torch.tensor([[0,0],[np.sqrt(g * N), 0]])
K = [K_1, K_2, K_3, K_4]

In [4]:
pms = process_matrices.ProcessMatrices(q_s, q_c, K)

In [10]:
optimizer = torch.optim.Adam(pms.parameters(), lr = 1e-5)

regularization = 10
loss = lambda X_C, X_E, X_R : - utils.avg_fidelity(X_C, X_E, X_R) + regularization * (utils.sums_to_identity(X_C) + utils.sums_to_identity(X_R))

In [24]:
l = loss(*pms())
l.backward()
optimizer.step()
l

tensor(4466290.4414, dtype=torch.float64, grad_fn=<AddBackward0>)

In [24]:
optimizer.step()